# AES 算法软件实现

## 实验目的

通过实验，掌握AES算法的加解密流程、特性和程序实现方法。理解分组密码的基本结构和步骤，了解SPN结构设计方法。

## 实验要求

- 编写`AES`算法加解密程序，下载烧写到 `OSR-Mega2560` 或 `OSR-407`开发板；
- 使用串口助手或python [pyserial](https://pypi.org/project/pyserial/) 库，与开发板进行通信，如下图所示：
<img src='../images/AES通信模型.png' width=300px/>

- 使用python [Crypto](https://pypi.org/project/pycryptodome/) 库，验证加解密的正确性；
- 评估实现的`AES`算法的加密效率和开发板资源使用情况。

## 实验环境需求

- **硬件需求**
    + OSR-Mega2560 或 OSR-407开发板
- **软件需求**
    + Arduino IDE
    + 串口助手

## 实验内容

### 实验原理

- **AES 算法背景**

  AES, Advanced Encryption Standard。NIST (National INstitute of Standards and Technology) 在1997年9月12日公开征集更高效更安全的替代DES加密算法，第一轮共有15种算法入选，其中5种算法入围了决赛，分别是MARS，RC6，Rijndael，Serpent和Twofish。又经过3年的验证、评测及公众讨论之后Rijndael算法最终入选，成为AES标准[FIPS 197](http://csrc.nist.gov/publications/fips/fips197/fips-197.pdf)。
  
  Rijndael算法是由比利时学者Joan Daemen和Vincent Rijmen所提出的，算法的名字由两位作者的名字组合而成。Rijndael算法在设计时支持多种分组及密钥长度，介于128-256之间所有32的倍数均可，最小支持128位，最大256位，共25种组合。AES标准支持的分组大小固定为128位，密钥长度有3种选择：128位、192位及256位。

- **AES 算法结构**
  
  AES 算法是采用代换-置换网络（SPN，Substitution-permutation network）的分组密码算法。明文数据经过多次轮变换生成密文，每个轮次的转换操作由轮函数定义。轮函数根据轮密钥对数据进行不同的代换及置换等操作。
  + **轮状态**
    AES标准算法将128位的明文，以特定次序生成一个4x4的矩阵（每个元素是一个字节，8位），即初始状态矩阵。字节序排布按照从上到下，从左到右进行编排。例如，16个字节$\left[ b_{0},b_{1},...,b_{15} \right] $将被编排为:
    
    $$
    \left[
      \begin{matrix}
       b_0 & b_4 & b_8 & b_{12}\\
       b_1 & b_5 & b_9 & b_{13}\\
       b_2 & b_6 & b_{10} & b_{14} \\
       b_3 & b_7 & b_{11} & b_{15}
      \end{matrix}
      \right]
    $$
    
  + **算法描述**
    1. 密钥扩展，根据密钥生成函数，由128/192/256位密钥生成轮函数所使用的轮密钥
    2. 初始化密钥加，将轮密钥与初始状态异或
    3. 9/11/13 轮
        - 字节代换(SubByte) 
        - 行移位(ShiftRow)
        - 列混合(MixColumn)
        - 轮密钥加(AddRoundKey)
    4. 最后一轮
        - 字节代换(SubByte) 
        - 行移位(ShiftRow)
        - 轮密钥加(AddRoundKey)
        
  + **字节代换(SubByte)**
      字节代换（SubBytes）是对状态矩阵中的每一个元素到S盒中进行查找并以此替换输入状态的操作。字节代换是可逆的非线性变换，也是AES运算中唯一的非线性变换。字节代换逆操作是通过逆S盒的查找及替换来完成的。
   
   ```c
   static uint8_t s_box[256] = {
        // 0     1     2     3     4     5     6     7     8     9     a     b     c     d     e     f
        0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, // 0
        0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, // 1
        0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, // 2
        0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, // 3
        0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84, // 4
        0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, // 5
        0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8, // 6
        0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, // 7
        0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73, // 8
        0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, // 9
        0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, // a
        0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, // b
        0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, // c
        0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, // d
        0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, // e
        0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16};// f

    static uint8_t inv_s_box[256] = {
        // 0     1     2     3     4     5     6     7     8     9     a     b     c     d     e     f
        0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb, // 0
        0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb, // 1
        0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e, // 2
        0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25, // 3
        0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92, // 4
        0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84, // 5
        0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06, // 6
        0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b, // 7
        0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73, // 8
        0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e, // 9
        0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b, // a
        0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4, // b
        0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f, // c
        0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef, // d
        0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61, // e
        0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d};// f
      ```
      
  + **行移位(ShiftRow)**
      对于状态矩阵，第一行不做变化，第二行向左循环移动一个元素，第三行向左循环移动两个元素，第三行向左循环移动三个元素。
  
  + **列混合(MixColumn)**
      对状态矩阵的每一列，列混合可以视作如下矩阵乘法操作：
      $$
      \left[
      \begin{matrix}
       b_{0j}\\
       b_{1j}\\
       b_{2j}\\
       b_{3j}
      \end{matrix}
      \right] = 
      \left[
      \begin{matrix}
       02 & 03 & 01 & 01\\
       01 & 02 & 03 & 01\\
       01 & 01 & 02 & 03 \\
       03 & 01 & 01 & 02
      \end{matrix}
      \right]
      \left[
      \begin{matrix}
       a_{0j}\\
       a_{1j}\\
       a_{2j}\\
       a_{3j}
      \end{matrix}
       \right]
    $$
    其中，元素乘法为$GF(2^8)$上的乘法运算，不可约多项式为$m(x) = x^8+x^4+x^3+x+1$。
  + **轮密钥加(AddRoundKey)** 将轮密钥与状态矩阵异或。
  
   更详细的算法加解密描述及测试向量请参照[NIST.FIPS.197](../docs/NIST.FIPS.197.pdf)。

### 实验过程

1. 在Arduino IDE中补全AES算法加密程序，参考代码见 [./AES/aes.c] [./AES/aes.ino]
2. 尝试将程序下载到开发板
3. 参考Python Crypto库的[参考文档](https://pycryptodome.readthedocs.io/en/latest/src/cipher/aes.html)，定义AES类，实现AES加解密，验证开发板的加解密结果是否正确
    - 注：安装[pycryptodome](https://pypi.org/project/pycryptodome/)将获得和pycrypto相同的接口，避免安装过程中的编译错误

### 实验代码

#### 与 TOE 进行通信

In [15]:
# 补全引用库
import serial
import numpy as np
from Crypto.Cipher import AES

In [16]:
# 补全函数参数
mega = serial.Serial('COM4', 115200, timeout=1)

In [17]:
def get_meta():
    # 产生随机明文，补全函数参数
    p = np.random.randint(0, 256, 16, dtype=np.uint8)
    send = p.tobytes()
    # 发送数据到开发板，补全函数
    mega.write(send)
    # 读取加密结果，补全函数
    recv = mega.read(16)
    c = np.frombuffer(recv, dtype=np.uint8)
    return p, c

#### 验证AES开发板加密结果是否正确

In [18]:
# 与开发板设置相同的密钥，验证开发板结果
key = bytes([0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00,])

In [19]:
aes = AES.new(key, AES.MODE_ECB)

In [20]:
def verify(p, c):
    # 补全函数
    c_correct = aes.encrypt(p.tobytes())
    print("c_correct: " , c_correct)
    print("c:", c.tobytes())
    if c.tobytes() == c_correct:
        return True
    else:
        return False

In [21]:
# 得到随机明文和开发板的加密结果
p, c = get_meta()

result = verify(p, c)
print("验证结果:", result)

c_correct:  b'\xe4\xe6\n\x1aF{\x1bkPkF\xb9l\xef\xact'
c: b'\xe4\xe6\n\x1aF{\x1bkPkF\xb9l\xef\xact'
验证结果: True


#### 在开发板循环迭代执行100、1000、10000次加密，记录加密结果和加密效率

In [32]:
import serial
import numpy as np
from Crypto.Cipher import AES
import time

In [33]:
mega = serial.Serial('COM4', 115200, timeout=1)

def init():
    # 产生随机明文，补全函数参数
    p = np.random.randint(0, 256, 16, dtype=np.uint8)
    send = p.tobytes()
    # 发送数据到开发板，补全函数
    mega.write(send)
    # 读取加密结果，补全函数
    recv = mega.read(16)
    c = np.frombuffer(recv, dtype=np.uint8)
    return p, c


def send_and_receive(p):
    mega.write(p.tobytes())
    recv = mega.read(16)
    c = np.frombuffer(recv, dtype=np.uint8)
    return c

def benchmark(iterations):
    start = time.time()
    print(start)
    _, c = init()
    p = c
    for _ in range(1, iterations):
        c = send_and_receive(p)
        p = c
    end = time.time()
    print(end)
    print(f"执行{iterations}次加密，耗时: {end - start:.2f}秒")

In [34]:
# 测试不同次数的加密效率
benchmark(100)

1729499441.7944474
1729499442.6105576
执行100次加密，耗时: 0.82秒


In [35]:
benchmark(1000)

1729499443.8112543
1729499452.0981057
执行1000次加密，耗时: 8.29秒


In [36]:
benchmark(10000)

1729499457.6223881
1729499540.2151327
执行10000次加密，耗时: 82.59秒
